In [ ]:
#Instalando dependencias Spark visto o desáfio estar sendo feito no Google Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=6e117e1aa93373c194857c8d28b66b6cbaa72579859383a69d8c6ca8718f3c09
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
import wget

wget.download('ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz')
wget.download('ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz')

'NASA_access_log_Aug95.gz'

In [ ]:
!pip install gunzip

In [ ]:
import gzip
import shutil
with gzip.open('NASA_access_log_Jul95.gz', 'rb') as f_in:
    with open('NASA_access_01', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
with gzip.open('NASA_access_log_Aug95.gz', 'rb') as f_in:
    with open('NASA_access_02', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
from pyspark import SparkContext, SQLContext, SparkConf, StorageLevel
conf = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '4'), 
                                   ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
rdd_1 = sc.textFile("NASA_access_01")
rdd_2 = sc.textFile("NASA_access_02")

In [ ]:
rdd = rdd_1.union(rdd_2)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

In [ ]:
rdd_split1 = rdd.map(lambda x: x.split(" "))

In [ ]:
rdd_split2 = rdd_split1.repartition(16)

In [ ]:
month_lst = [('Jan','01'), ('Feb','02'), ('Mar','03'), ('Apr','04'), ('May','05'), ('Jun','06'), ('Jul','07'), ('Aug','08'), ('Sep','09'), ('Oct','10'), ('Nov','11'), ('Dec','12')]

def ajustMonth(monthname):
  for x in month_lst:
    if monthname == x[0]:
      return x[1]
      break
  return "nil"    

Py4JJavaError: ignored

In [ ]:
rdd_split3 = rdd_split2.filter(lambda x: len(x) > 9)
rdd_split3.cache()

In [ ]:
rdd_split4 = rdd_split3.map(lambda x: (x[0], x[3].replace("[", "")[0:20], x[8], x[9]))

In [ ]:
rdd_split5 = rdd_split4.map(lambda x: (x[0], x[1][0:3]+ajustMonth(x[1][3:6])+x[1][6:11]+' '+x[1][12:20], x[2], x[3]))

In [ ]:
rdd_split5.take(4)

Total partitions: 4


In [ ]:
df = rdd_split5.toDF(["Host", "DataRequest", "StatusCode", "Bytes"])

AttributeError: ignored

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_formatdata = df.select(col("Host"), col("StatusCode"), col("Bytes"), to_timestamp(date_format(to_utc_timestamp(
  to_timestamp(col("DataRequest"), "dd/MM/yyyy HH:mm:ss"),"America/Argentina/Buenos_Aires"), "dd-MM-yyyy HH:mm:ss"),"dd-MM-yyyy HH:mm:ss").alias("DataRequest"))

df_formatbytes = df_formatdata.withColumn("Bytes", df_formatdata["bytes"].cast(IntegerType()))

In [ ]:
df_formatbytes.schema

In [ ]:
df_formatbytes.createOrReplaceTempView("request")

In [ ]:
#Host Distintos

spark.sql("""
  SELECT DISTINCT host FROM request
""").show()

In [ ]:
#total de erros 404 no periodo especificado

spark.sql("""
  SELECT count(1) as registros FROM request
  WHERE
  StatusCode = '404' AND
  DataRequest BETWEEN '1995-07-01 03:00:59' AND '1995-07-10 03:00:00'
""").show()

In [ ]:
#Quais dias do período especificado tiveram o maior número de erros 404.

spark.sql("""
  SELECT date_format(DataRequest,"dd-MM-yyyy") as DataRequest, COUNT(1) as total FROM request
  WHERE
  StatusCode = '404' AND
  DataRequest BETWEEN '1995-07-01 03:00:59' AND '1995-07-10 03:00:00'
  GROUP BY date_format(DataRequest,"dd-MM-yyyy")
  ORDER BY count(1) DESC
  LIMIT 5
""").show()

In [ ]:
#O total de bytes retornados no período, com uma visão acumulada. Por exemplo, se no
#dia 1 tivemos 50 bytes, dia 2 tivemos 100 bytes e dia 3 mais 150 bytes, sua resposta
#deverá ser: Dia 1 = 50 bytes, Dia 2 = 150 bytes, Dia 3 = 300 bytes.

spark.sql("""
  WITH temptable (DataRequest, Bytes) AS
  (
    SELECT date_trunc("Day",DataRequest) AS DataRequest, SUM(bytes) AS Bytes FROM request
    GROUP BY date_trunc("Day",DataRequest)
  )
  SELECT 
  DataRequest,
  Bytes,
  sum(Bytes) OVER (ORDER BY DataRequest) as total
  FROM temptable
  ORDER BY DataRequest;
""").show()